<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_016_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Exercise 0
#!pip install -q biopython
from Bio import SeqIO
import requests
from io import StringIO

Prot_IDs=['P04406','P10599','P68871','P04637']
def getFasta(ID):
    url = f"https://rest.uniprot.org/uniprotkb/{ID}.fasta"
    response = requests.get(url)
    response.raise_for_status()
    fasta_data = StringIO(response.text)

    for seq_record in SeqIO.parse(fasta_data, "fasta"):
      print(len(seq_record.seq))
      return str(seq_record.seq)


query_dict={}
for id in Prot_IDs:
  query_dict[id]=getFasta(id)

print(query_dict)


335
sp|P04406|G3P_HUMAN
105
sp|P10599|THIO_HUMAN
147
sp|P68871|HBB_HUMAN
393
sp|P04637|P53_HUMAN
{'P04406': 'MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVKAENGKLVINGNPITIFQERDPSKIKWGDAGAEYVVESTGVFTTMEKAGAHLQGGAKRVIISAPSADAPMFVMGVNHEKYDNSLKIISNASCTTNCLAPLAKVIHDNFGIVEGLMTTVHAITATQKTVDGPSGKLWRDGRGALQNIIPASTGAAKAVGKVIPELNGKLTGMAFRVPTANVSVVDLTCRLEKPAKYDDIKKVVKQASEGPLKGILGYTEHQVVSSDFNSDTHSSTFDAGAGIALNDHFVKLISWYDNEFGYSNRVVDLMAHMASKE', 'P10599': 'MVKQIESKTAFQEALDAAGDKLVVVDFSATWCGPCKMIKPFFHSLSEKYSNVIFLEVDVDDCQDVASECEVKCMPTFQFFKKGQKVGEFSGANKEKLEATINELV', 'P68871': 'MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH', 'P04637': 'MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSG

In [19]:
#Exercise 1
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML


recordlst=[]
for key,value in query_dict.items():
  record=SeqRecord(Seq(value),id=key)
  recordlst.append(record)
with open('query_proteins.fasta', "w") as output:
  queryfasta = SeqIO.write(recordlst,output, "fasta")

with open("blast_results.xml") as f:
    blast_records = list(NCBIXML.parse(f))  # one record per query protein

alignment_dict={}
for record in blast_records:
    print(f"\n{record.query}")

    query_id = record.query.split(' ')[0]
    hits = {}
    for rank, alignment in enumerate(record.alignments[:10]):
        hsp = alignment.hsps[0]
        uid = alignment.hit_def.split('|')[1]
        hits[uid] = {
            'blast_rank': rank,
            'e_value':    hsp.expect,
            'pct_id':     hsp.identities / hsp.align_length,
        }
        alignment_dict[query_id]=hits
print(alignment_dict)



P04406 <unknown description>

P10599 <unknown description>

P68871 <unknown description>

P04637 <unknown description>
{'P04406': {'P0A9B2': {'blast_rank': 0, 'e_value': 1.40553e-161, 'pct_id': 0.6606060606060606}, 'P0A9B6': {'blast_rank': 1, 'e_value': 1.24916e-82, 'pct_id': 0.375}}, 'P10599': {'P0AA25': {'blast_rank': 0, 'e_value': 7.02746e-14, 'pct_id': 0.3522727272727273}, 'P0AGG4': {'blast_rank': 1, 'e_value': 1.07126e-13, 'pct_id': 0.3235294117647059}, 'P77395': {'blast_rank': 2, 'e_value': 2.36368e-06, 'pct_id': 0.25274725274725274}}}


In [15]:
#Exercise 2
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import h5py

def compute_similarity(query_embeddings,db_embeddings):

  query_matrix=np.array([query_embeddings[id] for id in Prot_IDs])

  db_ids=list(db_embeddings.keys())
  db_matrix=np.array([db_embeddings[id] for id in db_ids])

  comp=cosine_similarity(query_matrix,db_matrix)
  return comp

def getProteinName(hit_id):
  for record in SeqIO.parse('ecoli.fasta', "fasta"):
    if hit_id in str(record.id):
      return str(record.id).split('|')[2]

def tophits(comparison,sequences,db_embeddings):
  query_order=Prot_IDs
  db_ids=list(db_embeddings.keys())

  result=[]

  for query_idx,id in enumerate(query_order):
    hits=np.argsort(comparison[query_idx])[::-1][:10].tolist()

    for i, idx in enumerate(hits):
      hit_id=db_ids[idx]
      similarity=comparison[query_idx][idx]
      sequence=sequences[hit_id]

      name=getProteinName(hit_id)

      result.append({
          'query': id,
          'rank': i,
          'ref_id': hit_id,
          'name': name,
          'similarity': similarity,
          'sequence': sequence
    })
  return result

with h5py.File('query_proteins.h5', 'r') as file:
    query_embeddings={}
    for name in file.keys():
        query_embeddings[name]=file[name][:]
with h5py.File('per-protein.h5', 'r') as file:
  db_embeddings={}
  for name in file.keys():
    db_embeddings[name]=file[name][:]

similarities=compute_similarity(query_embeddings,db_embeddings)

sequences={}
for record in SeqIO.parse('ecoli.fasta', 'fasta'):
  parse=record.id.split('|')
  if len(parse)>=2:
    uniprot_id=parse[1]
  else:
    uniprot_id=parse[0]
  sequences[uniprot_id]=str(record.seq)

results=tophits(similarities,sequences,db_embeddings)
resultdf=pd.DataFrame(results)
display(resultdf)

,query,rank,ref_id,name,similarity,sequence
0,P04406,0,P0A9B2,G3P1_ECOLI,0.927050,MTIKVGINGFGRIGRIVFRAAQKRSDIEIVAINDLLDADYMAYMLK...
1,P04406,1,P61889,MDH_ECOLI,0.821558,MKVAVLGAAGGIGQALALLLKTQLPSGSELSLYDIAPVTPGVAVDL...
2,P04406,2,P0CE48,EFTU2_ECOLI,0.808739,MSKEKFERTKPHVNVGTIGHVDHGKTTLTAAITTVLAKTYGGAARA...
3,P04406,3,P0CE47,EFTU1_ECOLI,0.808287,MSKEKFERTKPHVNVGTIGHVDHGKTTLTAAITTVLAKTYGGAARA...
4,P04406,4,P0AGE9,SUCD_ECOLI,0.777293,MSILIDKNTKVICQGFTGSQGTFHSEQAIAYGTKMVGGVTPGKGGT...
5,P04406,5,P0A6P9,ENO_ECOLI,0.759750,MSKIVKIIGREIIDSRGNPTVEAEVHLEGGFVGMAAAPSGASTGSR...
6,P04406,6,P08200,IDH_ECOLI,0.752024,MESKVVVPAQGKKITLQNGKLNVPENPIIPYIEGDGIGVDVTPAML...
7,P04406,7,P0A817,METK_ECOLI,0.746620,MAKHLFTSESVSEGHPDKIADQISDAVLDAILEQDPKARVACETYV...
8,P04406,8,P0A6F5,CH60_ECOLI,0.746416,MAAKDVKFGNDARVKMLRGVNVLADAVKVTLGPKGRNVVLDKSFGA...
9,P04406,9,P0A6A6,LEUC_ECOLI,0.743258,MAKTLYEKLFDAHVVYEAENETPLLYIDRHLVHEVTSPQAFDGLRA...


In [18]:
#Exercise 3
from scipy.stats import spearmanr
def emb_top10(resultdf):
    output = {}
    for query_id, grp in resultdf.groupby('query'):
        output[query_id] = {row['ref_id']: int(row['rank']) for _, row in grp.iterrows()}
    return output

emb_dict   = emb_top10(resultdf)
summary_rows = []

for query_id in alignment_dict:
        blast_ids = set(alignment_dict[query_id].keys())
        emb_ids   = set(emb_dict.get(query_id, {}).keys())

        intersection = blast_ids & emb_ids
        union        = blast_ids | emb_ids
        overlap      = len(intersection)
        jaccard      = overlap / len(union) if union else 0.0

        # Rank correlation for shared hits
        spearman_r = np.nan
        p_val      = np.nan
        if len(intersection) >= 3:          # need at least 3 points
            shared = sorted(intersection)
            b_ranks = [alignment_dict[query_id][uid]['blast_rank'] for uid in shared]
            e_ranks = [emb_dict[query_id][uid]                 for uid in shared]
            res = spearmanr(b_ranks, e_ranks)
            spearman_r, p_val = res.statistic, res.pvalue

        print(f"{query_id:<12} {overlap:>4}/10   {jaccard:>7.3f}   {spearman_r:>10.3f}   {p_val:>9.4f}")

        # Per-protein detailed breakdown
        print(f"\n  Shared proteins ({overlap}):")
        for uid in sorted(intersection):
            br = alignment_dict[query_id][uid]['blast_rank']
            er = emb_dict[query_id].get(uid, '–')
            ev = alignment_dict[query_id][uid]['e_value']
            pi = alignment_dict[query_id][uid]['pct_id']
            print(f"    {uid}  BLAST rank {br+1:>2}  |  Emb rank {er+1 if isinstance(er,int) else er:>2}  "
                  f"|  E-val {ev:.2e}  |  %ID {pi:.1%}")

        blast_only = blast_ids - emb_ids
        emb_only   = emb_ids   - blast_ids
        if blast_only:
            print(f"  BLAST-only ({len(blast_only)}): {', '.join(sorted(blast_only))}")
        if emb_only:
            print(f"  Embedding-only ({len(emb_only)}): {', '.join(sorted(emb_only))}")
        print()

        summary_rows.append({
            'query':        query_id,
            'blast_hits':   len(blast_ids),
            'emb_hits':     len(emb_ids),
            'overlap':      overlap,
            'jaccard':      round(jaccard, 3),
            'spearman_r':   round(spearman_r, 3) if not np.isnan(spearman_r) else 'N/A',
            'p_value':      round(p_val, 4)       if not np.isnan(p_val)      else 'N/A',
        })

summary_df = pd.DataFrame(summary_rows)
display(summary_df)

P04406          1/10     0.091          nan         nan

  Shared proteins (1):
    P0A9B2  BLAST rank  1  |  Emb rank  1  |  E-val 1.41e-161  |  %ID 66.1%
  BLAST-only (1): P0A9B6
  Embedding-only (9): P08200, P0A6A6, P0A6F5, P0A6P9, P0A817, P0AGE9, P0CE47, P0CE48, P61889

P10599          2/10     0.182          nan         nan

  Shared proteins (2):
    P0AA25  BLAST rank  1  |  Emb rank  1  |  E-val 7.03e-14  |  %ID 35.2%
    P0AGG4  BLAST rank  2  |  Emb rank  2  |  E-val 1.07e-13  |  %ID 32.4%
  BLAST-only (1): P77395
  Embedding-only (8): P0ABQ2, P0AC69, P0AC81, P0AE08, P0AE52, P0AEQ3, P61949, P69441



,query,blast_hits,emb_hits,overlap,jaccard,spearman_r,p_value
0,P04406,2,10,1,0.091,N/A,N/A
1,P10599,3,10,2,0.182,N/A,N/A


In [ ]:
#Exercise 4